<a href="https://colab.research.google.com/github/laresamdeola/Natural-Language-Processing/blob/main/Pretrained_Bert_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import time

In [10]:
torch.cuda.is_available()

True

In [11]:
!pip install accelerate

In [12]:
!pip install datasets transformers huggingface_hub

In [13]:
#!pip install transformers huggingface_hub
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [14]:
from datasets import Dataset, load_dataset, DatasetDict

In [15]:
data = pd.read_csv('Random_2000.csv')

In [16]:
data_annotated = data[:1112]

In [17]:
type(data_annotated)

pandas.core.frame.DataFrame

In [18]:
train_data = data_annotated[:556]
test_data = data_annotated[556:]

In [19]:
type(train_data)

pandas.core.frame.DataFrame

In [20]:
train_data.to_csv('train_data_2000.csv', index=False)
time.sleep(30)

In [21]:
test_data.to_csv('test_data_2000.csv', index=False)

In [22]:
data_files = {'train': 'train_data_2000.csv', 'test': 'test_data_2000.csv'}

In [23]:
random_2000_dataset = load_dataset('csv', data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [24]:
random_2000_dataset

DatasetDict({
    train: Dataset({
        features: ['Comment', 'SentimentalLabel'],
        num_rows: 562
    })
    test: Dataset({
        features: ['Comment', 'SentimentalLabel'],
        num_rows: 559
    })
})

In [25]:
random_2000_dataset = random_2000_dataset.filter(lambda x: x['Comment'] is not None)

Filter:   0%|          | 0/562 [00:00<?, ? examples/s]

Filter:   0%|          | 0/559 [00:00<?, ? examples/s]

In [26]:
random_2000_dataset = random_2000_dataset.filter(lambda x: x['SentimentalLabel'] is not None)

Filter:   0%|          | 0/560 [00:00<?, ? examples/s]

Filter:   0%|          | 0/553 [00:00<?, ? examples/s]

In [27]:
random_2000_dataset

DatasetDict({
    train: Dataset({
        features: ['Comment', 'SentimentalLabel'],
        num_rows: 554
    })
    test: Dataset({
        features: ['Comment', 'SentimentalLabel'],
        num_rows: 548
    })
})

In [28]:
small_train_dataset = random_2000_dataset["train"].shuffle(seed=42).select([i for i in list(range(500))])
small_test_dataset = random_2000_dataset["test"].shuffle(seed=42).select([i for i in list(range(500))])
print(small_train_dataset[0])
print(small_test_dataset[0])

{'Comment': 'hi bro can you recreate the inventions of nikola tesla whose words are being hidden by america or someone for their own sake and as far as i know i39ve read that there are also inventions of nikola tesla such as wireless charging like the example you gave i hope in this world there are people who can make new discoveries or restore old inventions that may be really needed in this day and age to save electricity', 'SentimentalLabel': 'Pos'}
{'Comment': 'as i understand it combustion vehicles generally have an energy efficiency of 20 let s consider the opportunity costs even if we only went with the 40 efficient charging system and even if only half the extra grid energy were to come from clean sources and even if none of the vehicles had a solar panel roof an inductively charged road network would still yield a net reduction in emissionsbonuses  what emissions continued to happen would do so in large remote static facilities that can have massive filters as opposed to gridl

In [29]:
len(small_test_dataset)

500

In [30]:
small_train_dataset[:5]

{'Comment': ['hi bro can you recreate the inventions of nikola tesla whose words are being hidden by america or someone for their own sake and as far as i know i39ve read that there are also inventions of nikola tesla such as wireless charging like the example you gave i hope in this world there are people who can make new discoveries or restore old inventions that may be really needed in this day and age to save electricity',
  'check this out',
  'the man who pioneerd 21st century with others like einstein father of ac and lasersome dude with 3 braincells quothe didnt understand his devicequot',
  'quottesla did not uderstud his invetions  quot are u mad  this tesla coil is not right tesla coil where are longitudual waves in you constraction it is wrong that u thing that is original tesla coil   stupid man',
  'tesla absolutely knew what he was doing people like thomas edison and j p morgan worked on destroying tesla credibility however his inventions will be distribute all through o

In [31]:
type(small_train_dataset)

datasets.arrow_dataset.Dataset

In [32]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [33]:
def preprocess_function(examples):
  return tokenizer(examples['Comment'], truncation=True)

In [34]:
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [35]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [58]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
from datasets import load_metric

def compute_metrics(eval_pred):
  load_accuracy = load_metric('accuracy')
  load_f1 = load_metric('f1')

  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  accuracy = load_accuracy.compute(predictions=predictions, references=labels)['accuracy']
  f1 = load_f1.compute(predictions=predictions, references=labels)['f1']
  return {'accuracy': accuracy, 'f1': f1}

In [50]:
from huggingface_hub import notebook_login

notebook_login()

In [51]:
!git config --global credential.helper store

In [62]:
# I'M HERE!!!!!!!

# Define a new Trainer with all the objects we constructed so far
from transformers import TrainingArguments, Trainer

repo_name = "laredeola/finetuning-sentiment-model-wireless-electricity-samples"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True,
    label_names=['SentimentalLabel']
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [63]:
trainer.train()

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`SentimentalLabel` in this case) have excessive nesting (inputs type `list` where type `int` is expected).